In [8]:
import csv
import requests
import json
import os
import pandas as pd
import sqlite3

In [6]:
# Set dir
local_dir = "data/"

url = 'https://prices.runescape.wiki/api/v1/osrs/24h'

# User agent tag used to let wiki know why I am using their API. 
userAgent = {
    'User-Agent': 'Machine Learning Practice',
    'From': 'sliverwall (discord ID)'  # discord ID
}

In [7]:
'''Grab 24h data'''

daily_file = local_dir + "24h.csv"
## Fetch data
jsonResponse = requests.get(url, headers=userAgent)
jsonData = jsonResponse.json()

# Extract data
currenttimestamp = jsonData["timestamp"]
responseData = jsonData["data"]

# Turn dict components to lists
keylist = list(responseData.keys())
valuelist = list(responseData.values())

totalvalues = []
totalkeysandvalues = []
for value in valuelist:
    subvalues = list(value.values())
    totalvalues.append(subvalues)
for key1, value1 in zip(keylist, totalvalues):
    totalkeysandvalues.append([key1] + value1)
with open(daily_file, 'w', newline='') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL, quotechar="'")
    writer.writerow(['id','avgHighPrice','highPriceVolume','avgLowPrice','lowPriceVolume'])
    writer.writerows(totalkeysandvalues)

In [10]:
'''create a template for generating reports'''

# read daily report as df
dailyCSV = pd.read_csv(daily_file, sep=",")

# Initialize temp sqlite db and import Pandas Dataframe
tempdb = sqlite3.connect(":memory:")

# Import Pandas Dataframe into temp sqlite db
dailyCSV.to_sql('dailyCSV', tempdb, if_exists='append', index=False)
cur = tempdb.cursor()

## run your queries here
cur.execute('''CREATE TABLE FinalOutput AS SELECT * FROM dailyCSV;''')

## print results to terminal
finalOutputReport = pd.read_sql('''SELECT * FROM finalOutput;''', tempdb)
print(finalOutputReport)

## export results to .csv
finalOutputReport.to_csv('HighLowSpreadReport.csv', index=False)

         id  avgHighPrice  highPriceVolume  avgLowPrice  lowPriceVolume
0         2         292.0         20955146        286.0         8200778
1         6      200309.0               89     193944.0             322
2         8      188577.0              146     185168.0             440
3        10      181042.0              178     175926.0             443
4        12      195983.0              181     188748.0             364
...     ...           ...              ...          ...             ...
3825  30616       14309.0             2989      12306.0            1280
3826  30619       11562.0             5010       9633.0            1786
3827  30628     3816619.0               11    3730358.0             154
3828  30631     2966663.0               26    2845450.0             142
3829  30634     6922334.0              594    6837116.0             420

[3830 rows x 5 columns]


In [17]:
## run your queries here
SpreadReport = pd.read_sql('''

SELECT id, avgHighPrice, (avgHighPrice - avgLowPrice) AS Spread, ((avgHighPrice - avgLowPrice) * ((lowPriceVolume + highPriceVolume)/2)) AS PotentialDailyGrossProfit 
FROM DailyCSV 
ORDER BY PotentialDailyGrossProfit DESC
''', tempdb)

print(SpreadReport)

         id  avgHighPrice     Spread  PotentialDailyGrossProfit
0     20997  1.510641e+09  4962157.0               1.141296e+09
1     27277  1.174168e+09  3629522.0               1.030784e+09
2     22486  1.821881e+09  5400907.0               8.479424e+08
3     26374  3.381085e+08  2038680.0               7.135380e+08
4     28338  4.466836e+08  3197190.0               6.586211e+08
...     ...           ...        ...                        ...
3825  29210           NaN        NaN                        NaN
3826  29213           NaN        NaN                        NaN
3827  29622           NaN        NaN                        NaN
3828  30328  9.900000e+03        NaN                        NaN
3829  30414           NaN        NaN                        NaN

[3830 rows x 4 columns]
